In [1]:
import pandas as pd
import numpy as np
import time
from missing_data_tests import littles_mcar_test, logistic_regression_test
from imputation import regression_imputation
import requests
import re

Libraries imported successfully!


# Data Collection and Data Storage

### Loading Data

In [2]:
movies = pd.read_csv('../DataSets/movies.csv')
links = pd.read_csv('../DataSets/links.csv')
ratings = pd.read_csv('../DataSets/ratings.csv')
tags = pd.read_csv('../DataSets/tags.csv')

These datasets are taken from  https://grouplens.org/datasets/movielens/latest website.


1. movies.csv - Movie metadata (movieId, title, genres)
2. ratings.csv - User ratings (userId, movieId, rating, timestamp)
3. tags.csv - User-generated tags (userId, movieId, tag, timestamp)
4. links.csv - External database links (movieId, imdbId, tmdbId)









In [3]:
print(f"Movies_shape - {movies.shape}")
print(f"Links_shape - {links.shape}")
print(f"Ratings_shape - {ratings.shape}")
print(f"Tags_shape -{tags.shape}")

Movies_shape - (9742, 3)
Links_shape - (9742, 3)
Ratings_shape - (100836, 4)
Tags_shape -(3683, 4)


In [4]:
links.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

In [5]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [6]:
tags.columns

Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')

* The *movies* dataset has 3 columns — movieId (unique identifier for each movie), title, and genres, containing details of *9,742 movies*.
* The *links* dataset has 3 columns — movieId, imdbId (identifier in IMDb database), and tmdbId (identifier in TMDb database), providing external references for *9,742 movies*.
* The *ratings* dataset has 4 columns — userId (identifier for each user), movieId, rating (rating given by that particular user), and timestamp (time at which the rating was given, recorded in UNIX format), containing *100,836 ratings* from *610 users*.
* The *tags* dataset has 4 columns — userId, movieId, tag (short description or label assigned by user), and timestamp (time when the tag was added), containing *3,683 user-generated tags*.

### Folder Structure

In [7]:
print("""
DA2402-PROJECT/
│
├── DataSets/
│   ├── links.csv
│   ├── movies.csv
│   ├── ratings.csv
│   └── tags.csv
|
├── DataPreProcessing/
│   ├── PreProcessedData/
│   │   ├── cleaned_data.csv
│   │   ├── movie_level_data.csv
│   │   └── scraped_data.csv
│   │
│   ├── Data_preprocessing.ipynb
│   ├── imputation.py
│   └── missing_data_tests.py
│
│
├── Visualization/
│   ├── Visualization_dashboard.py
│   └── Visualization.ipynb
│
├── DA2402_Project_Movie_Ratings_Analysis.pdf
└── README.txt
""")


DA2402-PROJECT/
│
├── DataSets/
│   ├── links.csv
│   ├── movies.csv
│   ├── ratings.csv
│   └── tags.csv
|
├── DataPreProcessing/
│   ├── PreProcessedData/
│   │   ├── cleaned_data.csv
│   │   ├── movie_level_data.csv
│   │   └── scraped_data.csv
│   │
│   ├── Data_preprocessing.ipynb
│   ├── imputation.py
│   └── missing_data_tests.py
│
│
├── Visualization/
│   ├── Visualization_dashboard.py
│   └── Visualization.ipynb
│
├── DA2402_Project_Movie_Ratings_Analysis.pdf
└── README.txt



## Merging of Files

In [8]:
movie_details = pd.merge(movies, links, on='movieId', how='left')

user_interactions = pd.merge(ratings, tags,
                             on=['userId', 'movieId'],
                             how='outer',
                             suffixes=('_rating', '_tag')) # Renames the 'timestamp' columns

merged_df = pd.merge(user_interactions, movie_details, on='movieId', how='left')

merged_df

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId
0,1,1,4.0,9.649827e+08,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,3,4.0,9.649812e+08,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
2,1,6,4.0,9.649822e+08,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0
3,1,47,5.0,9.649838e+08,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0
4,1,50,5.0,9.649829e+08,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0
...,...,...,...,...,...,...,...,...,...,...
102879,610,166534,4.0,1.493848e+09,NaN,NaN,Split (2017),Drama|Horror|Thriller,4972582,381288.0
102880,610,168248,5.0,1.493850e+09,Heroic Bloodshed,1.493844e+09,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0
102881,610,168250,5.0,1.494273e+09,NaN,NaN,Get Out (2017),Horror,5052448,419430.0
102882,610,168252,5.0,1.493846e+09,NaN,NaN,Logan (2017),Action|Sci-Fi,3315342,263115.0


# Data Cleaning

### Handle Duplicate values

In [9]:
cleaned_df = merged_df.dropna(subset=['userId']).copy()
cleaned_df


,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId
0,1,1,4.0,9.649827e+08,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,3,4.0,9.649812e+08,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
2,1,6,4.0,9.649822e+08,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0
3,1,47,5.0,9.649838e+08,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0
4,1,50,5.0,9.649829e+08,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0
...,...,...,...,...,...,...,...,...,...,...
102879,610,166534,4.0,1.493848e+09,NaN,NaN,Split (2017),Drama|Horror|Thriller,4972582,381288.0
102880,610,168248,5.0,1.493850e+09,Heroic Bloodshed,1.493844e+09,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0
102881,610,168250,5.0,1.494273e+09,NaN,NaN,Get Out (2017),Horror,5052448,419430.0
102882,610,168252,5.0,1.493846e+09,NaN,NaN,Logan (2017),Action|Sci-Fi,3315342,263115.0


In [10]:
duplicate_criteria = ['title', 'userId']

cleaned_df = cleaned_df.drop_duplicates(subset=duplicate_criteria, keep='first').copy()

In [11]:
cleaned_df

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId
0,1,1,4.0,9.649827e+08,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,3,4.0,9.649812e+08,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
2,1,6,4.0,9.649822e+08,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0
3,1,47,5.0,9.649838e+08,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0
4,1,50,5.0,9.649829e+08,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0
...,...,...,...,...,...,...,...,...,...,...
102879,610,166534,4.0,1.493848e+09,NaN,NaN,Split (2017),Drama|Horror|Thriller,4972582,381288.0
102880,610,168248,5.0,1.493850e+09,Heroic Bloodshed,1.493844e+09,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0
102881,610,168250,5.0,1.494273e+09,NaN,NaN,Get Out (2017),Horror,5052448,419430.0
102882,610,168252,5.0,1.493846e+09,NaN,NaN,Logan (2017),Action|Sci-Fi,3315342,263115.0


### Extracting Release Year of movie

In [12]:
cleaned_df['year'] = (
    cleaned_df['title']
    .str.extractall(r'(\b(18|19|20)\d{2}\b)')[0]   # extract all possible 4-digit years
    .groupby(level=0).last()                      # take the last one per row
    .astype(float)
)
cleaned_df

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year
0,1,1,4.0,9.649827e+08,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995.0
1,1,3,4.0,9.649812e+08,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995.0
2,1,6,4.0,9.649822e+08,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0,1995.0
3,1,47,5.0,9.649838e+08,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0,1995.0
4,1,50,5.0,9.649829e+08,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0,1995.0
...,...,...,...,...,...,...,...,...,...,...,...
102879,610,166534,4.0,1.493848e+09,NaN,NaN,Split (2017),Drama|Horror|Thriller,4972582,381288.0,2017.0
102880,610,168248,5.0,1.493850e+09,Heroic Bloodshed,1.493844e+09,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0,2017.0
102881,610,168250,5.0,1.494273e+09,NaN,NaN,Get Out (2017),Horror,5052448,419430.0,2017.0
102882,610,168252,5.0,1.493846e+09,NaN,NaN,Logan (2017),Action|Sci-Fi,3315342,263115.0,2017.0


### Handle missing values

In [13]:
cleaned_df[cleaned_df['rating'].isna()]

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year
3287,21,1569,NaN,NaN,romantic comedy,1.419805e+09,My Best Friend's Wedding (1997),Comedy|Romance,119738,8874.0,1997.0
3634,21,118985,NaN,NaN,painter,1.419805e+09,Big Eyes (2014),Drama,1126590,87093.0,2014.0
18869,119,260,NaN,NaN,classic,1.435943e+09,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,76759,11.0,1977.0
28476,193,7361,NaN,NaN,alternate reality,1.435857e+09,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,338013,38.0,2004.0
43443,288,7020,NaN,NaN,Notable Nudity,1.150988e+09,Proof (1991),Comedy|Drama|Romance,102721,14904.0,1991.0
...,...,...,...,...,...,...,...,...,...,...,...
90538,573,4448,NaN,NaN,boring,1.186589e+09,"Score, The (2001)",Action|Drama,227445,11371.0,2001.0
90562,573,6016,NaN,NaN,not seen,1.186589e+09,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller,317248,598.0,2002.0
90563,573,6157,NaN,NaN,bad,1.186589e+09,Daredevil (2003),Action|Crime,287978,9480.0,2003.0
97196,600,273,NaN,NaN,gothic,1.237739e+09,Mary Shelley's Frankenstein (Frankenstein) (1994),Drama|Horror|Sci-Fi,109836,3036.0,1994.0


In [14]:
# convert type of data in year column from str to int
cleaned_df['year'] = pd.to_numeric(cleaned_df['year'], errors='coerce')
cleaned_df['year'].dtype


dtype('float64')

## MCAR on Ratings

In [15]:
numeric_cols = ['rating', 'year', 'userId']

# Test on MCAR dataset
d2_mcar, p_value_mcar, df_mcar_test = littles_mcar_test(cleaned_df, numeric_cols)
print("Little's MCAR Test on MCAR dataset:")
print(f"Test statistic: {d2_mcar:.4f}")
print(f"Degrees of freedom: {df_mcar_test}")
print(f"p-value: {p_value_mcar:.4f}")
print(f"Conclusion: {'Data are MCAR (fail to reject H0)' if p_value_mcar > 0.05 else 'Data are not MCAR (reject H0)'}")

        rating  year  userId
0            0     0       0
1            0     0       0
2            0     0       0
3            0     0       0
4            0     0       0
...        ...   ...     ...
102879       0     0       0
102880       0     0       0
102881       0     0       0
102882       0     0       0
102883       0     0       0

[100972 rows x 3 columns]
{(0, 0, 0): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], (0, 1, 0): [9458, 17212, 18205, 18207, 30462, 30513, 47022, 47140, 50308, 58534, 65458, 71148, 72704, 82744, 84679, 89812, 97123], (1, 0, 0): [3287, 3634, 18869, 28476, 43443, 43600, 49727, 49813, 49880, 52303, 52869

## MAR on ratings

In [16]:
predictors = ['userId', 'year','movieId']
print("Logistic Regression Test on MCAR dataset:")
_, significant_predictors_mcar = logistic_regression_test(cleaned_df, 'rating', predictors)
print(f"Testing if 'income' missingness can be predicted by observed variables:")
print(f"Significant predictors: {significant_predictors_mcar}")
print(f"Conclusion: {'Data are MAR' if significant_predictors_mcar else 'Data are MCAR'}")

Logistic Regression Test on MCAR dataset:
Testing if 'income' missingness can be predicted by observed variables:
Significant predictors: ['userId', 'year']
Conclusion: Data are MAR


## Imputation in Ratings Columns using Regression

In [17]:
col_to_impute = 'rating'
predictor_cols = ['movieId', 'userId'] # 'movieId' is first, 'userId' is second

# Make sure your original DataFrame is named 'df' or change it here
try:
    cleaned_df = regression_imputation(cleaned_df, col_to_impute, predictor_cols)

    print(f"\nSuccessfully imputed missing values in '{col_to_impute}' using hierarchical regression imputation.")

except NameError:
    print("\n--- Error: DataFrame 'df' not found. ---")
    print("Please make sure your DataFrame is loaded into a variable named 'df'.")
except KeyError as e:
    print(f"\n--- Error: Column not found: {e} ---")
    print("Please check that 'df' contains 'rating', 'movieId', and 'userId' columns.")

--- Starting hierarchical imputation for 'rating' ---
Missing values BEFORE imputation: 140

Step 1: Calculating movieId level average...
Missing values after Step 1 (movieId): 18

Step 2: Calculating userId level average...
Missing values after Step 2 (userId): 0

--- Imputation Complete ---
Original missing: 140
Final missing:    0

Successfully imputed missing values in 'rating' using hierarchical regression imputation.


In [18]:
movie_means = cleaned_df.groupby('movieId')['rating'].transform('mean')
cleaned_df['rating'] = cleaned_df['rating'].fillna(movie_means)
cleaned_df[cleaned_df['rating'].isna()]

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year,missing_indicator


In [19]:
cleaned_df.dropna(subset=['rating'], inplace=True)


## Missing values in year and genre

In [20]:
cleaned_df[cleaned_df['year'].isnull()].head()


,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year,missing_indicator
9458,62,176601,5.0,1.525795e+09,NaN,NaN,Black Mirror,(no genres listed),2492564,452830.0,NaN,0
17212,105,147250,5.0,1.526207e+09,NaN,NaN,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),229922,127605.0,NaN,0
18205,111,171631,1.0,1.517441e+09,NaN,NaN,Maria Bamford: Old Baby,(no genres listed),6264596,455601.0,NaN,0
18207,111,171891,3.5,1.517440e+09,NaN,NaN,Generation Iron 2,(no genres listed),6263642,447818.0,NaN,0
30462,209,140956,3.5,1.524522e+09,NaN,NaN,Ready Player One,Action|Sci-Fi|Thriller,1677720,333339.0,NaN,0


In [21]:
len(cleaned_df[cleaned_df['year'].isnull()])

17

In [22]:
cleaned_df[cleaned_df['genres'] == '(no genres listed)'].head()

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year,missing_indicator
3642,21,122896,4.0,1.452052e+09,NaN,NaN,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),1790809,166426.0,2017.0,0
7373,50,114335,3.0,1.522446e+09,NaN,NaN,La cravate (1957),(no genres listed),121731,32891.0,1957.0,0
7428,50,174403,2.5,1.527542e+09,NaN,NaN,The Putin Interviews (2017),(no genres listed),6840134,461805.0,2017.0,0
9342,62,122896,3.5,1.523048e+09,NaN,NaN,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),1790809,166426.0,2017.0,0
9448,62,172591,5.0,1.525637e+09,NaN,NaN,The Godfather Trilogy: 1972-1990 (1992),(no genres listed),150742,364150.0,1992.0,0


In [23]:
len(cleaned_df[cleaned_df['genres'] == '(no genres listed)'])

47

## Scraping of year and genres from  wikepedia

In [24]:
def get_movie_extract(movie_title):
    """Search Wikipedia for a movie and return its extract text."""
    url = "https://en.wikipedia.org/w/api.php"
    headers = {"User-Agent": "MovieGenreFinder/1.0 (contact: vvenkatasreekar@gmail.com)"}

    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": f"{movie_title} film",
        "format": "json"
    }

    try:
        search_response = requests.get(url, params=search_params, headers=headers, timeout=10)
        search_response.raise_for_status()
        search_data = search_response.json()
    except requests.exceptions.RequestException as e:
        return f"Request error: {e}"

    search_results = search_data.get("query", {}).get("search", [])
    if not search_results:
        return "No extract found."

    best_title = search_results[0]["title"]

    extract_params = {
        "action": "query",
        "format": "json",
        "titles": best_title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
        "redirects": 1
    }

    try:
        extract_response = requests.get(url, params=extract_params, headers=headers, timeout=10)
        extract_response.raise_for_status()
        extract_data = extract_response.json()
    except requests.exceptions.RequestException as e:
        return f"Request error: {e}"

    pages = extract_data.get("query", {}).get("pages", {})
    page = next(iter(pages.values()), {})
    extract = page.get("extract", "No extract found.")
    return extract


def detect_genre(text):
    """Detect genre(s) from the Wikipedia extract text."""
    text = text.lower()

    genre_patterns = {
        "science fiction": [r"science fiction", r"sci[\s\-]?fi"],
        "action": [r"action"],
        "drama": [r"drama"],
        "thriller": [r"thriller"],
        "romantic": [r"romantic", r"romance"],
        "comedy": [r"comedy", r"comedies"],
        "fantasy": [r"fantasy"],
        "horror": [r"horror"],
        "documentary": [r"documentary"],
        "animation": [r"animation", r"animated"],
        "adventure": [r"adventure"],
        "mystery": [r"mystery"],
        "crime": [r"crime"],
        "biography": [r"biographical", r"biography", r"biopic"]
    }

    first_two_sentences = '.'.join(text.split('.')[:2])
    found = []

    for genre, patterns in genre_patterns.items():
        for pattern in patterns:
            if re.search(r"\b" + pattern + r"\b", first_two_sentences):
                found.append(genre)
                break

    return found


def find_year(text):
    """Extract release year (1800–2099) from Wikipedia extract."""
    match = re.search(r'\b(18|19|20)\d{2}\b', text)
    return int(match.group(0)) if match else None


# --- Test Example ---
text = get_movie_extract("The Godfather Trilogy: 1972-1990 (1992)")
print("Extract:\n", text[:300], "...\n")  # print first 300 chars only
print("Genres:", detect_genre(text))
print("Year:", find_year(text))


Extract:
 The Godfather is a trilogy of American crime films directed by Francis Ford Coppola inspired by the 1969 novel of the same name by Italian American author Mario Puzo. The films follow the trials of the fictional Italian American mafia Corleone family whose patriarch, Vito Corleone, rises to be a maj ...

Genres: ['crime']
Year: 1969


In [25]:
def fill_missing_info(df):
    """Fill missing genres (marked as '(no genres listed)') and missing years using Wikipedia data."""
    genre_cache = {}
    year_cache = {}

    # Identify rows that actually need data
    mask_missing = df['genres'].isin([None, '', '(no genres listed)']) | df['year'].isna()
    missing_titles = df.loc[mask_missing, 'title'].unique()

    print(f" Found {len(missing_titles)} unique titles needing info.\n")

    for title in missing_titles:
        print(f"Fetching Wikipedia info for: {title}")
        extract = get_movie_extract(title)
        if extract == "No extract found.":
            continue

        genre_list = detect_genre(extract)
        year = find_year(extract)

        genre_cache[title] = ', '.join(genre_list) if genre_list else None
        year_cache[title] = year

        # Delay to avoid rate limiting (Wikipedia API request rate)
        time.sleep(1)

    # Update only missing genre and year rows
    mask_genre = df['genres'].isin([None, '', '(no genres listed)'])
    df.loc[mask_genre, 'genres'] = df.loc[mask_genre, 'title'].map(genre_cache)

    mask_year = df['year'].isna()
    df.loc[mask_year, 'year'] = df.loc[mask_year, 'title'].map(year_cache)

    print("\n Finished updating missing genres and years.")
    return df

# cleaned_df = fill_missing_info(cleaned_df)

In [26]:
# cleaned_df = cleaned_df.drop(columns=['missing_indicator'])
# cleaned_df = cleaned_df.dropna(subset=['genres']).copy()
# cleaned_df.to_csv("PreProcessedData/scraped_data.csv")

In [27]:
user_level_df = pd.read_csv("PreProcessedData/scraped_data.csv")

In [28]:
user_level_df

,Unnamed: 0,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year
0,0,1,1,4.0,9.649827e+08,NaN,NaN,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995.0
1,1,1,3,4.0,9.649812e+08,NaN,NaN,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995.0
2,2,1,6,4.0,9.649822e+08,NaN,NaN,Heat (1995),Action|Crime|Thriller,113277,949.0,1995.0
3,3,1,47,5.0,9.649838e+08,NaN,NaN,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0,1995.0
4,4,1,50,5.0,9.649829e+08,NaN,NaN,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0,1995.0
...,...,...,...,...,...,...,...,...,...,...,...,...
100951,102879,610,166534,4.0,1.493848e+09,NaN,NaN,Split (2017),Drama|Horror|Thriller,4972582,381288.0,2017.0
100952,102880,610,168248,5.0,1.493850e+09,Heroic Bloodshed,1.493844e+09,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0,2017.0
100953,102881,610,168250,5.0,1.494273e+09,NaN,NaN,Get Out (2017),Horror,5052448,419430.0,2017.0
100954,102882,610,168252,5.0,1.493846e+09,NaN,NaN,Logan (2017),Action|Sci-Fi,3315342,263115.0,2017.0


In [29]:
user_level_df.isnull().sum()

Unnamed: 0              0
userId                  0
movieId                 0
rating                  0
timestamp_rating      140
tag                 99181
timestamp_tag       99181
title                   0
genres                  0
imdbId                  0
tmdbId                 13
year                    0
dtype: int64

### UNIX timestamps to readable date-time

In [30]:
user_level_df['timestamp_rating'] = pd.to_datetime(user_level_df['timestamp_rating'], unit='s')
user_level_df['timestamp_tag'] = pd.to_datetime(user_level_df['timestamp_tag'], unit='s')


In [31]:
# Pandas will automatically detect the string format YYYY-MM-DD HH:MM:SS
user_level_df['timestamp_rating'] = pd.to_datetime(user_level_df['timestamp_rating'])
user_level_df['timestamp_tag'] = pd.to_datetime(user_level_df['timestamp_tag'])
user_level_df

,Unnamed: 0,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,year
0,0,1,1,4.0,2000-07-30 18:45:03,NaN,NaT,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,NaN,NaT,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,NaN,NaT,Heat (1995),Action|Crime|Thriller,113277,949.0,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,NaN,NaT,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,NaN,NaT,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0,1995.0
...,...,...,...,...,...,...,...,...,...,...,...,...
100951,102879,610,166534,4.0,2017-05-03 21:53:22,NaN,NaT,Split (2017),Drama|Horror|Thriller,4972582,381288.0,2017.0
100952,102880,610,168248,5.0,2017-05-03 22:21:31,Heroic Bloodshed,2017-05-03 20:44:30,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0,2017.0
100953,102881,610,168250,5.0,2017-05-08 19:50:47,NaN,NaT,Get Out (2017),Horror,5052448,419430.0,2017.0
100954,102882,610,168252,5.0,2017-05-03 21:19:12,NaN,NaT,Logan (2017),Action|Sci-Fi,3315342,263115.0,2017.0


### Calculate average ratings and rating counts per movie

In [32]:
average_ratings_df = user_level_df.groupby('movieId')['rating'].mean().reset_index()
average_ratings_df = average_ratings_df.rename(columns={'rating': 'average_rating'})

rating_counts_df = user_level_df.groupby('movieId')['rating'].count().reset_index()
rating_counts_df = rating_counts_df.rename(columns={'rating': 'rating_count'})

movie_level_df = user_level_df.drop_duplicates(subset=['movieId']).copy()

movie_level_df = movie_level_df[['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'year']]

movie_level_df = pd.merge(movie_level_df, average_ratings_df, on='movieId', how='left')
movie_level_df = pd.merge(movie_level_df, rating_counts_df, on='movieId', how='left')
movie_level_df

,movieId,title,genres,imdbId,tmdbId,year,average_rating,rating_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995.0,3.920930,215
1,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995.0,3.259615,52
2,6,Heat (1995),Action|Crime|Thriller,113277,949.0,1995.0,3.946078,102
3,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0,1995.0,3.975369,203
4,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0,1995.0,4.237745,204
...,...,...,...,...,...,...,...,...
9719,160341,Bloodmoon (1997),Action|Thriller,118745,30948.0,1997.0,2.500000,1
9720,160527,Sympathy for the Underdog (1971),Action|Crime|Drama,66806,90351.0,1971.0,4.500000,1
9721,160836,Hazard (2005),Action|Drama|Thriller,798722,70193.0,2005.0,3.000000,1
9722,163937,Blair Witch (2016),Horror|Thriller,1540011,351211.0,2016.0,3.500000,1


### Split pipe-separated genres into individual categories

In [33]:
import ast
import pandas as pd

movies_df = movie_level_df.copy()  # or your dataframe

# --- Step 1: Ensure genres column is a list ---
def ensure_list(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            # Handle both list-form and pipe-form strings
            return ast.literal_eval(x) if x.startswith("[") else [g.strip() for g in x.replace("|", ",").split(",")]
        except Exception:
            return [g.strip() for g in x.replace("|", ",").split(",")]
    return []

movies_df["genres"] = movies_df["genres"].apply(ensure_list)

# --- Step 2: Normalize genre names before one-hot encoding ---
def normalize_genre(g):
    g = g.strip().lower().replace("-", " ")
    mapping = {
        "sci fi": "Science Fiction",
        "scifi": "Science Fiction",
        "science fiction": "Science Fiction",
        "sciencefiction": "Science Fiction",
    }
    return mapping.get(g, g.title())  # title-case for normal names

movies_df["genres"] = movies_df["genres"].apply(lambda lst: [normalize_genre(g) for g in lst])

# --- Step 3: One-hot encode the cleaned genres ---
genres_onehot = (
    movies_df["genres"]
    .explode()
    .dropna()
    .str.get_dummies()
    .groupby(level=0)
    .sum()
    .clip(0, 1)
)

# --- Step 4: Remove old columns that conflict ---
movies_df = movies_df.drop(columns=[c for c in genres_onehot.columns if c in movies_df.columns], errors="ignore")

# --- Step 5: Join one-hot columns back ---
movies_df = movies_df.join(genres_onehot)

# --- Step 6: Save back ---
movie_level_df = movies_df



In [34]:
movie_level_df.columns

Index(['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'year',
       'average_rating', 'rating_count', 'Action', 'Adventure', 'Animation',
       'Biography', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film Noir', 'Horror', 'Imax', 'Musical', 'Mystery',
       'Romance', 'Science Fiction', 'Thriller', 'War', 'Western'],
      dtype='object')

In [35]:
movie_level_df.to_csv("PreProcessedData/movie_level_data.csv", index=False)

## Removing lookalike genres from the feature.

In [36]:
import ast
import pandas as pd

# Copy user_level_df into updated_df
updated_df = user_level_df.copy()

# --- Step 1: Ensure genres column is a list ---
def ensure_list(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            # Handle both list-form and pipe-form strings
            return ast.literal_eval(x) if x.startswith("[") else [g.strip() for g in x.replace("|", ",").split(",")]
        except Exception:
            return [g.strip() for g in x.replace("|", ",").split(",")]
    return []

updated_df["genres"] = updated_df["genres"].apply(ensure_list)

# --- Step 2: Normalize genre names before one-hot encoding ---
def normalize_genre(g):
    g = g.strip().lower().replace("-", " ")
    mapping = {
        "sci fi": "Science Fiction",
        "scifi": "Science Fiction",
        "science fiction": "Science Fiction",
        "sciencefiction": "Science Fiction",
    }
    return mapping.get(g, g.title())  # title-case for normal names

updated_df["genres"] = updated_df["genres"].apply(lambda lst: [normalize_genre(g) for g in lst])

# --- Step 3: One-hot encode the cleaned genres ---
genres_onehot = (
    updated_df["genres"]
    .explode()
    .dropna()
    .str.get_dummies()
    .groupby(level=0)
    .sum()
    .clip(0, 1)
)

# --- Step 4: Remove old columns that conflict ---
updated_df = updated_df.drop(columns=[c for c in genres_onehot.columns if c in updated_df.columns], errors="ignore")

# --- Step 5: Join one-hot columns back ---
updated_df = updated_df.join(genres_onehot)

# --- Step 6: Save back ---
user_level_df = updated_df



In [37]:
user_level_df.columns

Index(['Unnamed: 0', 'userId', 'movieId', 'rating', 'timestamp_rating', 'tag',
       'timestamp_tag', 'title', 'genres', 'imdbId', 'tmdbId', 'year',
       'Action', 'Adventure', 'Animation', 'Biography', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film Noir', 'Horror',
       'Imax', 'Musical', 'Mystery', 'Romance', 'Science Fiction', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [38]:
user_level_df.columns

Index(['Unnamed: 0', 'userId', 'movieId', 'rating', 'timestamp_rating', 'tag',
       'timestamp_tag', 'title', 'genres', 'imdbId', 'tmdbId', 'year',
       'Action', 'Adventure', 'Animation', 'Biography', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film Noir', 'Horror',
       'Imax', 'Musical', 'Mystery', 'Romance', 'Science Fiction', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [39]:
user_level_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [40]:
user_level_df.drop(82029, inplace = True)
user_level_df.drop(columns = 'Biography', inplace=True)

In [41]:
user_level_df

,userId,movieId,rating,timestamp_rating,tag,timestamp_tag,title,genres,imdbId,tmdbId,...,Film Noir,Horror,Imax,Musical,Mystery,Romance,Science Fiction,Thriller,War,Western
0,1,1,4.0,2000-07-30 18:45:03,NaN,NaT,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",114709,862.0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,2000-07-30 18:20:47,NaN,NaT,Grumpier Old Men (1995),"[Comedy, Romance]",113228,15602.0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,2000-07-30 18:37:04,NaN,NaT,Heat (1995),"[Action, Crime, Thriller]",113277,949.0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,2000-07-30 19:03:35,NaN,NaT,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]",114369,807.0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,2000-07-30 18:48:51,NaN,NaT,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]",114814,629.0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100951,610,166534,4.0,2017-05-03 21:53:22,NaN,NaT,Split (2017),"[Drama, Horror, Thriller]",4972582,381288.0,...,0,1,0,0,0,0,0,1,0,0
100952,610,168248,5.0,2017-05-03 22:21:31,Heroic Bloodshed,2017-05-03 20:44:30,John Wick: Chapter Two (2017),"[Action, Crime, Thriller]",4425200,324552.0,...,0,0,0,0,0,0,0,1,0,0
100953,610,168250,5.0,2017-05-08 19:50:47,NaN,NaT,Get Out (2017),[Horror],5052448,419430.0,...,0,1,0,0,0,0,0,0,0,0
100954,610,168252,5.0,2017-05-03 21:19:12,NaN,NaT,Logan (2017),"[Action, Science Fiction]",3315342,263115.0,...,0,0,0,0,0,0,1,0,0,0


In [42]:
user_level_df.index = range(len(user_level_df))

## Cleaned DataSet

In [43]:
user_level_df.to_csv('PreProcessedData/cleaned_data.csv', index=False)